# DNSSEC-Aware Resolver Downgrade Attacks

In [102]:
import logging
import random
import string
from datetime import datetime
import itertools
import concurrent
import math

import numpy as np
import dns.message, dns.query, dns.rdataclass, dns.rdatatype, dns.flags, dns.exception, dns.name, dns.dnssec
from tqdm import tqdm
import pandas as pd
import requests

IN = dns.rdataclass.from_text("IN")
NS = dns.rdatatype.from_text("NS")
SOA = dns.rdatatype.from_text("SOA")
DS = dns.rdatatype.from_text("DS")
A = dns.rdatatype.from_text("A")
TXT = dns.rdatatype.from_text("TXT")
AAAA = dns.rdatatype.from_text("AAAA")
RRSIG = dns.rdatatype.from_text("RRSIG")

ALGORITHMS = [
    dns.dnssec.RSASHA1,
    dns.dnssec.RSASHA256,
    dns.dnssec.RSASHA512,
    dns.dnssec.ECDSAP256SHA256,
    dns.dnssec.ECDSAP384SHA384,
    dns.dnssec.ED25519,
    dns.dnssec.ED448,
]

ZONE = dns.name.from_text('downgrade.dedyn.io')

executor = concurrent.futures.ThreadPoolExecutor(1)

def query(qname, resolver, cd, rdtype=A):
    q = dns.message.make_query(qname, rdtype)
    q.flags |= dns.flags.AD
    if cd:
        q.flags |= dns.flags.CD
    
    if resolver.startswith('https'):
        method = dns.query.https
        where = resolver
    elif resolver.startswith('tls'):
        method = dns.query.tls
        where = resolver[len('tls://'):]
    else:
        method = dns.query.udp
        where = resolver
        
    logging.info(f'Query:\n{q}')
    
    try:
        return method(q, where=where, timeout=2)
    except (dns.exception.Timeout, requests.exceptions.ReadTimeout, EOFError):
        return method(q, where=where, timeout=5)
    
def run(f, args_list):
    results = []    
    try:
        futures = [executor.submit(f, *args) for args in args_list]
        with tqdm(total=len(futures)) as pbar:
            for future in concurrent.futures.as_completed(futures):
                pbar.update(1)
                if future.exception():
                    logging.warning(f"{future.exception()}")
                    results.append({'status': future.exception()})
                else:
                    results.append(future.result())
    finally:
        return results    

## Define Test Zones with Different Combinations of DS and DNSKEY Records

In [103]:
ALGORITHMS_RED = ALGORITHMS = [
    dns.dnssec.RSASHA256,
    dns.dnssec.ECDSAP256SHA256,
    dns.dnssec.ED25519,
    dns.dnssec.ED448,
]

In [104]:
zones = [
    {
        'ds': algos, 
        'dnskey': tuple(sorted(set(algos) - set(remove_dnskeys))),
        'name': dns.name.from_text(
            "-".join(
                [f"ds{a}" for a in sorted(algos)] +
                [f"dnskey{int(a)}" for a in sorted(set(algos) - set(remove_dnskeys))]
            ),
            origin=ZONE
        ),
    }
    for algos in itertools.chain(itertools.combinations(ALGORITHMS, 1), itertools.combinations(ALGORITHMS_RED, 2))
    for remove_dnskeys in [[a for i, a in enumerate(algos) if v[i]] for v in itertools.product([True, False], repeat=len(algos))]
    #if 16 in algos
]
zones = pd.DataFrame(zones)
zones = zones.set_index('name')
zones

,ds,dnskey
name,,
"(b'ds8', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.RSASHA256,)",()
"(b'ds8-dnskey8', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.RSASHA256,)","(Algorithm.RSASHA256,)"
"(b'ds13', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ECDSAP256SHA256,)",()
"(b'ds13-dnskey13', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ECDSAP256SHA256,)","(Algorithm.ECDSAP256SHA256,)"
"(b'ds15', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ED25519,)",()
"(b'ds15-dnskey15', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ED25519,)","(Algorithm.ED25519,)"
"(b'ds16', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ED448,)",()
"(b'ds16-dnskey16', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ED448,)","(Algorithm.ED448,)"
"(b'ds8-ds13', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.RSASHA256, Algorithm.ECDSAP256SHA256)",()


## Define Resolvers to be Studied: In the Lab (UDP/TCP) and In the Wild (UDP/TCP, TLS, HTTPS)

In [105]:
open_resolvers = [{'resolver_addr': row['IPv4'], 'resolver_name': row['Handle'], 'resolver_group': 'open-named'} for _, row in pd.read_csv("open-resolvers.csv").iterrows()]
lab_resolvers = [{'resolver_addr': row['IPv4'], 'resolver_name': row['Handle'], 'resolver_group': 'lab'} for _, row in pd.read_csv("lab-resolvers.csv").iterrows()]

In [106]:
def convert_resolver_format(d):
    return [{'resolver_addr': addr, 'resolver_name': handle, 'resolver_group': 'open-named'} for handle, addr in d.items()]
    
#lab_resolvers = convert_resolver_format({})
#open_resolvers = convert_resolver_format({
#    #'quad9': '9.9.9.9',
#    #'google': '8.8.8.8',
#    'cloudflare': '1.1.1.1',
#    #'adguard': '94.140.14.14',
#})
doh_resolvers = convert_resolver_format({
    'cloudflare-doh': 'https://cloudflare-dns.com/dns-query',
    'cloudflare-mozilla-doh': 'https://mozilla.cloudflare-dns.com/dns-query',
    'google-doh': 'https://dns.google/dns-query',
    'quad9-doh': 'https://dns.quad9.net/dns-query',
    'clean-browsing-doh': 'https://security-filter-dns.cleanbrowsing.org/dns-query',
    'adguard-doh': 'https://dns.adguard.com/dns-query',
    'comcast-doh': 'https://doh.xfinity.com/dns-query',
})
dot_resolvers = convert_resolver_format({
    'cloudflare-dot': 'tls://1.1.1.1',
    'google-dot': 'tls://8.8.8.8',
    'quad9-dot': 'tls://9.9.9.9',
    'clean-browsing-dot': 'tls://185.228.168.9',
    'adguard-dot': 'tls://94.140.14.14',
})

In [107]:
def resolver_transport(row):
    if row['resolver_addr'].startswith('tls'):
        return 'DoT'
    if row['resolver_addr'].startswith('https'):
        return 'DoH'
    return 'UDP/TCP'

resolver_list = pd.DataFrame(
    open_resolvers + lab_resolvers + 
    doh_resolvers + dot_resolvers 
)
resolver_list['resolver_transport'] = resolver_list.apply(resolver_transport, axis=1)
resolver_list.head(5)

,resolver_addr,resolver_name,resolver_group,resolver_transport
0,208.67.222.222,cisco-umbrella,open-named,UDP/TCP
1,1.1.1.1,cloudflare,open-named,UDP/TCP
2,8.26.56.26,comodo-secure-dns,open-named,UDP/TCP
3,193.17.47.1,cznic-odvr,open-named,UDP/TCP
4,80.80.80.80,freenom-world,open-named,UDP/TCP


### Determine Resolver Cipher Support

In [108]:
def check_resolver(resolver, algorithm):
    try:
        qname = dns.name.from_text(f'mitm-ms.ds{algorithm}-dnskey{algorithm}', origin=ZONE)
        
        r = query(qname, resolver['resolver_addr'], cd=False, rdtype=TXT)  # signature invalid
        
        return {
            **resolver,
            'algorithm': algorithm,
            'status': 'ok',
            'qname': qname.to_text(),
            'ad': dns.flags.AD in r.flags,
            'rcode': r.rcode()
        }
    except (dns.exception.Timeout, requests.exceptions.ReadTimeout, EOFError):
        return {
            **resolver,
            'algorithm': algorithm,
            'status': 'timeout',
        }
    except Exception as e:
        return {
            **resolver,
            'algorithm': algorithm,
            'status': (type(e), e),
        }

In [109]:
logging.basicConfig(level=logging.WARNING, force=True)

In [110]:
resolver_support_results = run(check_resolver, [(resolver, a) for _, resolver in resolver_list.iterrows() for a in ALGORITHMS])
resolver_support = pd.DataFrame(resolver_support_results)

100%|██████████| 132/132 [00:50<00:00,  2.61it/s]


In [111]:
def support(row):
    def log():
        logging.warning(f'Weird resolver behavior for {row["resolver_name"]}: {row["qname1"]} -> {row["rcode1"]}, {row["qname2"]} -> {row["rcode2"]}')
        
    if row['status'] != 'ok':
        return None
    
    return row['rcode'] == dns.rcode.Rcode.SERVFAIL
    
resolver_support['supported'] = resolver_support.apply(support, axis=1)

In [112]:
def uncertain_any(s):
    if None in list(s):  # None in s is always false, likely due to pandas' messing with the 'in' operator
        return None
    else:
        return any(s)
    
grouped = resolver_support.groupby(['resolver_addr', 'algorithm'], dropna=False)[['supported']].agg({
    'supported': [uncertain_any]
}).reset_index().pivot(index='resolver_addr', columns='algorithm', values=('supported', 'uncertain_any')).reset_index()
grouped.columns = ['resolver_addr'] + [f'supports_{a}' for a in ALGORITHMS]
grouped['support'] = grouped.apply(lambda row: tuple(a for a in ALGORITHMS if row[f'supports_{a}'] is True), axis=1)
resolvers = grouped.set_index('resolver_addr').join(resolver_list.set_index('resolver_addr'))

In [113]:
resolvers.loc['9.9.9.9', 'supports_16'] = False
resolvers.loc['tls://9.9.9.9', 'supports_16'] = False
resolvers.loc['https://dns.quad9.net/dns-query', 'supports_16'] = False

In [114]:
def row_style(row):
    styles = {
        True: 'color: green;',
        False: 'color: red;',
    }
    return [styles.get(v) for v in row]
    
resolvers.style.apply(row_style, axis=1)

,supports_8,supports_13,supports_15,supports_16,support,resolver_name,resolver_group,resolver_transport
resolver_addr,,,,,,,,
1.1.1.1,True,True,True,False,"(, , )",cloudflare,open-named,UDP/TCP
141.12.174.11,True,True,False,False,"(, )",ws2012r2,lab,UDP/TCP
141.12.174.29,True,True,False,False,"(, )",bind9113,lab,UDP/TCP
141.12.174.30,True,True,False,False,"(, )",kresd532,lab,UDP/TCP
141.12.174.38,True,True,True,True,"(, , , )",powerdns460,lab,UDP/TCP
141.12.174.39,True,True,True,False,"(, , )",unbound167,lab,UDP/TCP
141.12.174.42,True,True,False,False,"(, )",ws2016,lab,UDP/TCP
141.12.174.44,True,True,True,True,"(, , , )",ws2012,lab,UDP/TCP
141.12.174.63,True,True,False,False,"(, )",ws2019,lab,UDP/TCP


In [115]:
resolvers['validating'] = resolvers.apply(lambda row: bool(row['support']), axis=1)
resolvers = resolvers[resolvers['validating']]

In [118]:
RESOLVER_NAMES = {
    'bind9113': 'Bind v9.11.3 (Ubuntu 18.04.3 LTS)',
    'kresd532': 'Knot Resolver 5.3.2',
    'powerdns460': 'Power DNS Recursor 4.6.0',
    'unbound167': 'Unbound 1.6.7',
    'ws2012': 'Windows Server 2012',
    'ws2012r2': 'Windows Server 2012 R2',
    'ws2016': 'Windows Server 2016',
    'ws2019': 'Windows Server 2019',
    'adguard': 'AdGuard Public DNS',
    'cloudflare': 'Cloudflare Resolver',
    'cloudflare-mozilla': 'Cloudflare Resolver for Mozilla',
    'comcast': 'Comcast Public DNS',
    'google': 'Google Public DNS',
    'quad9': 'Quad9 Resolver',
    'cisco-umbrella': 'Cisco Umbrella (former OpenDNS)',
    'comodo-secure-dns': 'Comodo Secure DNS',
    'cznic-odvr': 'cznic ODVR',
    'freenom-world': 'Freenom World',
    'oracle-dyn': 'Oracle Dyn',
    'yandex': 'Yandex safe'
}

RESOLVER_GROUPS = {
    'lab': 'Lab',
    'open-named': 'Public DNS',
}
ALGORITHM_NAMES = [f"{dns.dnssec.Algorithm.to_text(a)} ({str(int(a))})" for a in ALGORITHMS]

In [119]:
def single_value(s):
    assert len(s) == 1
    return s[0]

def removesuffix(s, suf):
    if s[-len(suf):] == suf:
        return s[:-len(suf)]
    return s

by = ['resolver_group', 'resolver_transport', 'resolver_name']  # resolver_addr
t = resolvers.sort_values(by).groupby(by).agg({
    f'supports_{a}': [single_value]
    for a in ALGORITHMS
}).reset_index()
t.columns = ['Group', 'Transport', 'Resolver'] + ALGORITHM_NAMES
formatters = {
    algorithm_name: lambda val: {True: r'\cmark', False: r'\xmark', None: '??'}[val]
    for algorithm_name in ALGORITHM_NAMES
}
formatters.update({
    'Group': lambda s: RESOLVER_GROUPS.get(s, s),
    'Resolver': lambda s: RESOLVER_NAMES.get(removesuffix(removesuffix(s, '-dot'), '-doh'), s),
})
print(t.to_latex(index=False, formatters=formatters, escape=False, column_format='lllcccc'))

\begin{tabular}{lllcccc}
\toprule
     Group & Transport &                          Resolver & RSASHA256 (8) & ECDSAP256SHA256 (13) & ED25519 (15) & ED448 (16) \\
\midrule
       Lab &   UDP/TCP & Bind v9.11.3 (Ubuntu 18.04.3 LTS) &        \cmark &               \cmark &       \xmark &     \xmark \\
       Lab &   UDP/TCP &               Knot Resolver 5.3.2 &        \cmark &               \cmark &       \xmark &     \xmark \\
       Lab &   UDP/TCP &          Power DNS Recursor 4.6.0 &        \cmark &               \cmark &       \cmark &     \cmark \\
       Lab &   UDP/TCP &                     Unbound 1.6.7 &        \cmark &               \cmark &       \cmark &     \xmark \\
       Lab &   UDP/TCP &               Windows Server 2012 &        \cmark &               \cmark &       \cmark &     \cmark \\
       Lab &   UDP/TCP &            Windows Server 2012 R2 &        \cmark &               \cmark &       \xmark &     \xmark \\
       Lab &   UDP/TCP &               Windows Server 

## Define Attack Strategies

In [120]:
attacks = [
    {'name': 'replace signature number with 253 (PRIVATEDNS) and fake content', 'instructions': ('rs17', 'at')},
    {'name': 'replace signature number with 17 (unassigned) and fake content', 'instructions': ('rs253', 'at')},
    {'name': 'replace signature number with ed448 and fake content', 'instructions': ('rs16', 'at')},
    {'name': 'replace signature number with ed25519 and fake content', 'instructions': ('rs15', 'at')},
    {'name': 'replace signature number with ecdsap256sha256 and fake content', 'instructions': ('rs13', 'at')},
    {'name': 'replace signature number with rsasha256 and fake content', 'instructions': ('rs8', 'at')},
    {'name': 'remove all signatures except ed448 and fake content', 'instructions': ('at',) + tuple(f'ds{a}' for a in ALGORITHMS if a < dns.dnssec.ED448)},
    {'name': 'remove all signatures except ed25519 and ed448 and fake content', 'instructions': ('at',) + tuple(f'ds{a}' for a in ALGORITHMS if a < dns.dnssec.ED25519)},
    {'name': 'strip all signatures and fake content', 'instructions': ('at',) + tuple(f'ds{a}' for a in ALGORITHMS)},
    {'name': 'invalidate signature', 'instructions': ('ms',),}
]
attacks = pd.DataFrame(attacks)
attacks['prefix'] = attacks.apply(lambda row: f"mitm-{'-'.join(row['instructions'])}", axis=1)
attacks = attacks.set_index('prefix')
attacks

,name,instructions
prefix,,
mitm-rs17-at,replace signature number with 253 (PRIVATEDNS) and fake content,"(rs17, at)"
mitm-rs253-at,replace signature number with 17 (unassigned) and fake content,"(rs253, at)"
mitm-rs16-at,replace signature number with ed448 and fake content,"(rs16, at)"
mitm-rs15-at,replace signature number with ed25519 and fake content,"(rs15, at)"
mitm-rs13-at,replace signature number with ecdsap256sha256 and fake content,"(rs13, at)"
mitm-rs8-at,replace signature number with rsasha256 and fake content,"(rs8, at)"
mitm-at-ds8-ds13-ds15,remove all signatures except ed448 and fake content,"(at, ds8, ds13, ds15)"
mitm-at-ds8-ds13,remove all signatures except ed25519 and ed448 and fake content,"(at, ds8, ds13)"
mitm-at-ds8-ds13-ds15-ds16,strip all signatures and fake content,"(at, ds8, ds13, ds15, ds16)"


## Run Attack Evaluation

In [16]:
def check_attack(addr, prefix, zone):
    try:
        qname = dns.name.from_text(prefix, origin=zone)
        r1 = query(qname, addr, cd=False, rdtype=TXT)
        logging.info(f'Response:\n{r1}')
        return {
            'resolver_addr': addr,
            'zone': zone,
            'attack': prefix,
            'status': 'ok',
            'rcode': r1.rcode(),
            'response': r1,
            'evil_content': 'evil' in str(r1) or 'ms-' in qname.to_text(),
        }
    except dns.exception.Timeout:
        return {
            'resolver_addr': addr,
            'zone': zone,
            'attack': prefix,
            'status': 'timeout',
        }
    except Exception as e:
        logging.warning(f"Exception: {type(e).__name__}: {e}")
        return {
            'resolver_addr': addr,
            'zone': zone,
            'attack': prefix,
            'status': e,
        }

In [ ]:
attack_results = run(check_attack, [(addr, prefix, zone) for prefix, _ in attacks.iterrows() for zone, _ in zones.iterrows() for addr, _ in resolvers.iterrows()])
attack_results = pd.DataFrame(attack_results)

  3%|▎         | 294/8640 [01:48<39:55,  3.48it/s]  

In [ ]:
results = attack_results.join(resolvers, on='resolver_addr').join(zones, on='zone').join(attacks, on='attack')

In [ ]:
assert len(attack_results) == len(results), (len(attack_results), len(results))

In [ ]:
results['status_str'] = results.apply(lambda row: str(row['status']), axis=1)
results['supported_ds'] = results.apply(lambda row: tuple(set(row['ds']) & set(row['support'])), axis=1)
results['supported_dnskey'] = results.apply(lambda row: tuple(set(row['dnskey']) & set(row['support'])), axis=1)
results['validation_paths'] = results.apply(lambda row: tuple(set(row['dnskey']) & set(row['ds'])), axis=1)
results['qname'] = results.apply(lambda row: dns.name.from_text(row['attack'], origin=row['zone']), axis=1)

In [ ]:
results['evil_content'] = results['qname'].apply(lambda x: '-ms' in x.to_text()) | results['evil_content']

In [ ]:
def behavior_correct(row):
    if row['status'] != 'ok':
        return None
    
    if not row['supported_ds']:
        return True
    
    if row['rcode'] == dns.rcode.Rcode.NOERROR and row['evil_content']:
        # signature invalid, but no error reported may only happen if no ds or no dnskey was supported
        return not row['supported_ds'] #or not row['supported_dnskey']
    elif row['rcode'] == dns.rcode.Rcode.SERVFAIL: # we don't see the content so cannot check for evil content:
        # desired behavior for invalid signatures
        # we don't care if resolvers give SERVFAIL more often than appropriate
        return True
    
    logging.warning(f"Don't know if behavior is correct for rcode={row['rcode']} evil_content={row['evil_content']} "
                    f"ds={', '.join(str(int(a)) for a in row['ds'])} "
                    f"supported_ds={', '.join(str(int(a)) for a in row['supported_ds'])} qname={row['qname']}")
    logging.warning(row['response'])
    
    return None

results['behavior_correct'] = results.apply(behavior_correct, axis=1)

In [ ]:
results['zone_prefix'] = results.apply(lambda row: row['zone'][0].decode(), axis=1)
results['zone_config'] = results.apply(lambda row: f"DS: {','.join(str(int(e)) for e in row['ds'])} DNSKEY: {','.join(str(int(e)) for e in row['dnskey'])}", axis=1)

## Show Success Attacks

In [ ]:
pd.options.display.max_rows = len(resolvers) * len(attacks)

def values(s):
    return '; '.join(s)

def zone_proportion(s):
    return len(s) / len(zones)

attack_success_rate = results.groupby(['attack', 'name', 'resolver_name', 'resolver_addr'], dropna=False).agg({
    'behavior_correct': [len, 'mean']
}).reset_index() #.style.apply(lambda row: ['', 'color: red' if row['behavior_correct']['mean'] < 1 else ''], axis=1)
attack_success_rate[attack_success_rate[('behavior_correct', 'mean')] < 1]

In [ ]:
pd.options.display.max_rows = len(resolvers) * len(attacks) * 2

def status_ok(s):
    return (s == 'ok').mean()

results.groupby(['resolver_group', 'resolver_name'], dropna=False).agg({
    'status': [status_ok],
    'behavior_correct': ['mean'],
})

In [ ]:
results['zone_name'] = results.apply(lambda row: row['zone'].to_text(), axis=1)

In [ ]:
results.to_pickle(datetime.now().strftime("results-%Y-%m-%d--%H-%M-%S.pickle"))

## Attack Prerequisites (**NOT** Sufficient Conditions!)

In [36]:
results = pd.read_pickle("results-2021-10-05--21-31-03.pickle")

In [40]:
pd.options.display.max_colwidth = 500

def infer_attack_setting(s):
    ret = []
    for a in ALGORITHMS:
        for rtype in ['ds']:
            if all(f"{rtype}{a}" in p for p in s):
                ret.append(f"{rtype}{a}")
            if all(f"{rtype}{a}" not in p for p in s):
                ret.append(f"NOT {rtype}{a}")
    for a1, a2 in itertools.combinations(ALGORITHMS, r=2):
        for rtype in ['ds']:
            if all(f"{rtype}{a1}" in p or f"{rtype}{a1}" in p for p in s):
                print('something happend', a1, a2)
                ret.append(f"({rtype}{a1} OR {rtype}{a2})")
    if not ret:
        return ', '.join(sorted(s))
    return ' and '.join(sorted(ret))

grouped = results[results['behavior_correct'] == False].groupby(['attack', 'resolver_name'], dropna=False).agg({
    'zone_prefix': [
        #values, 
        infer_attack_setting,
    ],
}).reset_index()
grouped.columns = grouped.columns.droplevel(1)
grouped = grouped.pivot(index=['attack'], columns=['resolver_name'])
grouped.columns = grouped.columns.droplevel(0)
grouped

resolver_name,adguard-doh,adguard-dot,cloudflare,cloudflare-doh,cloudflare-dot,cloudflare-mozilla-doh,google,google-doh,google-dot,quad9-doh,ws2012,ws2012r2,ws2016,ws2019
attack,,,,,,,,,,,,,,
mitm-at-ds8-ds13,"ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16","ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16",ds16,ds16,ds16,ds16,"ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16","ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds16-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16","ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds16-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16",NOT ds13 and NOT ds15 and NOT ds8 and ds16,"ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16"
mitm-at-ds8-ds13-ds15,ds16,ds16,ds16,ds16,ds16,ds16,ds16,ds16,ds16,NaN,"ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16"
mitm-at-ds8-ds13-ds15-ds16,NaN,NaN,ds16,ds16,ds16,ds16,NaN,NaN,NaN,NaN,"ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16"
mitm-ms,NaN,NaN,ds16,ds16,ds16,ds16,NaN,NOT ds13 and NOT ds15 and NOT ds8 and ds16,NOT ds13 and NOT ds15 and NOT ds8 and ds16,NaN,"ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16","ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16"
mitm-rs13-at,"ds13-ds15-dnskey15, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds8-dnskey8, ds8-ds13-dnskey8, ds8-ds15-dnskey15, ds8-ds15-dnskey8, ds8-ds15-dnskey8-dnskey15, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16","ds13-ds15-dnskey15, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds8-dnskey8, ds8-ds13-dnskey8, ds8-ds15-dnskey15, ds8-ds15-dnskey8, ds8-ds15-dnskey8-dnskey15, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16",ds16,ds16,ds16,ds16,"ds13-ds15-dnskey15, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds8-dnskey8, ds8-ds13-dnskey8, ds8-ds15-dnskey15, ds8-ds15-dnskey8, ds8-ds15-dnskey8-dnskey15, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16","ds13-ds15-dnskey15, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds16-dnskey16, ds8-dnskey8, ds8-ds13-dnskey8, ds8-ds15-dnskey15, ds8-ds15-dnskey8, ds8-ds15-dnskey8-dnskey15, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16","ds13-ds15-dnskey15, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds16-dnskey16, ds8-dnskey8, ds8-ds13-dnskey8, ds8-ds15-dnskey15, ds8

In [39]:
def unique_values(s):
    return sorted(set([x for x in s]))
    
grouped = results[results['behavior_correct'] == False].groupby(['attack', 'resolver_name'], dropna=False).agg({
    'zone_prefix': [
        unique_values, 
    ],
}).reset_index()
grouped.columns = grouped.columns.droplevel(1)
grouped = grouped.pivot(index=['attack'], columns=['resolver_name'])
grouped.columns = grouped.columns.droplevel(0)
grouped

resolver_name,adguard-doh,adguard-dot,cloudflare,cloudflare-doh,cloudflare-dot,cloudflare-mozilla-doh,google,google-doh,google-dot,quad9-doh,ws2012,ws2012r2,ws2016,ws2019
attack,,,,,,,,,,,,,,
mitm-at-ds8-ds13,"[ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13-dnskey16, ds13-ds16-dnskey16, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13-dnskey16, ds13-ds16-dnskey16, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13-dnskey16, ds13-ds16-dnskey16, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13-dnskey16, ds13-ds16-dnskey16, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16, ds8-ds16-dnskey8-dnskey16]","[ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds16-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds13-ds15-dnskey13, ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds16-dnskey16, ds8-ds15-dnskey8, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds16, ds16-dnskey16]","[ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16]","[ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16]","[ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16]","[ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16]"
mitm-at-ds8-ds13-ds15,"[ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13-dnskey16, ds13-ds16-dnskey16, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13-dnskey16, ds13-ds16-dnskey16, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13-dnskey16, ds13-ds16-dnskey16, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13-dnskey16, ds13-ds16-dnskey16, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds16-dnskey16, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]","[ds13-ds16-dnskey13, ds13-ds16-dnskey13-dnskey16, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds16-dnskey16, ds8-ds16-dnskey8, ds8-ds16-dnskey8-dnskey16]",NaN,"[ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds15-dnskey15, ds15-ds16-dnskey15, ds15-ds16-dnskey15-dnskey16, ds15-ds16-dnskey16, ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16]","[ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16]","[ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16]","[ds13-ds15-dnskey15, ds13-ds16-dnskey16, ds8-ds15-dnskey15, ds8-ds16-dnskey16]"
mitm-at-ds8-ds13-ds15-ds16,NaN,NaN,"[ds13-ds16-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16]","[ds13-ds16-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16]","[ds13-ds16-dnskey16, ds15-ds16-dnskey16, ds8-ds16-dnskey16]","[ds13-ds16-dnskey16, ds15-ds16-d

## Debug

In [ ]:
results[
    (results['resolver_name'] == 'adguard-doh') &
    (results['zone_prefix'] == 'ds13-dnskey13') &
    (results['attack'] == 'mitm-rs16-at')
]

In [ ]:
index = 5507

In [ ]:
print(results.loc[index]['response'])

In [ ]:
logging.basicConfig(level=logging.INFO, force=True)

In [ ]:
check_attack(results.loc[index]['resolver_addr'], results.loc[index]['attack'], dns.name.from_text(results.loc[index]['zone_prefix'], origin=ZONE))